## Elo Rating method

This is the Elo method.  The example is taken from "Who's #1" by Langville and Meyer and the results match.  Note, the data includes the entire 2009 NFL season, including all playoff games.  

Reference: https://www.gautamnarula.com/rating/

In [18]:
import math 
import pandas as pd
import numpy as np
  
# Function to calculate the Probability 
def Probability(rating1, rating2): 
  
    return 1.0 * 1.0 / (1 + 1.0 * math.pow(10, 1.0 * (rating1 - rating2) / 400)) 
  
# Function to calculate Elo rating 
# K is a constant. 
# Player A wins over Player B.  
# tie = true if tie, false otherwise
# home = 1 if player A is home, -1 if not, 0 if neutral
def EloRating(Ra, Rb, K, tie, home, homeWeight): 
    
    # To calculate the Winning 
    # Probability of Player B 
    Pb = Probability(Ra, Rb) 
  
    # To calculate the Winning 
    # Probability of Player A 
    Pa = Probability(Rb, Ra) 
  
    # Updating the Elo Ratings 
    if tie:
       Ra = Ra + K * (1/2 - Pa) 
       Rb = Rb + K * (1/2 - Pb) 
    else:        
       Ra = Ra + K * (1 - Pa) 
       Rb = Rb + K * (0 - Pb)
    if home == 1: 
        Ra = Ra+homeWeight
        Rb = Rb-homeWeight
    elif home == -1:
        Ra = Ra-homeWeight
        Rb = Rb+`1`1`1`1`1`1`1`1`homeWeight
        
    return Ra, Rb

### Create Elo ratings

In [19]:
def CalcEloRatings(K, homeWeight, games, numGames):
    eloRatings = np.zeros(numTeams)

    for i in range(numGames):
        team1ID = games.loc[i, 2] - 1 # subtracting 1 since python indexes at 0
        team1Score = games.loc[i, 4]
        team2ID = games.loc[i, 5] - 1 # subtracting 1 since python indexes at 0
        team2Score = games.loc[i, 7]

        if team1Score > team2Score:
            team1Rating, team2Rating = EloRating(eloRatings[team1ID], eloRatings[team2ID], K, False, games.loc[i, 3], homeWeight)
        elif team1Score < team2Score:
            team2Rating, team1Rating = EloRating(eloRatings[team2ID], eloRatings[team1ID], K, False, games.loc[i, 6], homeWeight)
        else:  
            team1Rating, team2Rating = EloRating(eloRatings[team1ID], eloRatings[team2ID], K, True, games.loc[i, 3], homeWeight)

        eloRatings[team1ID] = team1Rating
        eloRatings[team2ID] = team2Rating
        
    return eloRatings


In [20]:
#Brier score for playoffs and playoff elo adjustment
    
def PlayoffsBrierScore(teamEloRatings, playoffGames, numPlayoffGames, K, homeWeight, 
                       Round1_K, Round2_K, Round3_K):
    TotBrierScore = 0
    numBrierScore = 0
    for i in range(numPlayoffGames):
        team1ID = playoffgames.loc[i, 2] - 1 
        team1Score = playoffgames.loc[i, 4]
        team2ID = playoffgames.loc[i, 5] - 1 
        team2Score = playoffgames.loc[i, 7]

        if team1Score > team2Score:
            TotBrierScore = TotBrierScore + ((1-Probability(teamEloRatings[team2ID], teamEloRatings[team1ID]))**2)
            numBrierScore += 1
            if i < 4:
                team1Rating, team2Rating = EloRating(teamEloRatings[team1ID], teamEloRatings[team2ID], Round1_K, False, playoffgames.loc[i, 3], homeWeight)
            if i < 8:
                team1Rating, team2Rating = EloRating(teamEloRatings[team1ID], teamEloRatings[team2ID], Round2_K, False, playoffgames.loc[i, 3], homeWeight)
            if i < 10:
                team1Rating, team2Rating = EloRating(teamEloRatings[team1ID], teamEloRatings[team2ID], Round3_K, False, playoffgames.loc[i, 3], homeWeight)
                
        elif team2Score > team1Score:
            TotBrierScore = TotBrierScore + ((1-Probability(teamEloRatings[team1ID], teamEloRatings[team2ID]))**2)
            numBrierScore += 1
            if i < 4:
                team2Rating, team1Rating = EloRating(teamEloRatings[team2ID], teamEloRatings[team1ID], Round1_K, False, playoffgames.loc[i, 6], homeWeight)
            if i < 8:
                team2Rating, team1Rating = EloRating(teamEloRatings[team2ID], teamEloRatings[team1ID], Round2_K, False, playoffgames.loc[i, 6], homeWeight)
            if i < 10:
                team2Rating, team1Rating = EloRating(teamEloRatings[team2ID], teamEloRatings[team1ID], Round3_K, False, playoffgames.loc[i, 6], homeWeight)
                
        if i < 10:
            teamEloRatings[team1ID] = team1Rating
            teamEloRatings[team2ID] = team2Rating
    AvgBrierScore = TotBrierScore/numBrierScore
    return AvgBrierScore
    
def AddBrierScoreData(AvgBrierScore, BrierScoreData, K, homeWeight, Round1_K, Round2_K, Round3_K):
    BrierScoreData.loc[len(BrierScoreData.index)] = [AvgBrierScore, K, homeWeight, Round1_K, Round2_K, Round3_K]
    #print(f'Brier Score: {AvgBrierScore:.4f}' + f' K: {K}' + f' homeWeight: {homeWeight}') 

#BrierScore()

In [35]:
import itertools

teamFilename = 'data/NFLteams.txt'
teamNames = pd.read_csv(teamFilename, header = None)
numTeams = len(teamNames)

BrierScoreData = pd.DataFrame(data = {'brier': [], 'K': [], 'homeWeight': [], 'Round1_K': [], 'Round2_K': [], 'Round3_K': []})
for k, k1, k2, k3 in itertools.product(range(65,70,1), range(65,70,1), range(65,70,1), range(65,70,1)):
    for w in itertools.product(range(75,80,1)):               
        totYearBrierScore = 0;
        yearCount = 0;
        for year in range(2013,2019):
            gameFilename = f'data/{year}NFLseason.txt'
            playoffgameFilename = f'data/{year}NFLplayoffs.txt'
            games = pd.read_csv(gameFilename, header = None)
            numGames = len(games)
            playoffgames = pd.read_csv(playoffgameFilename, header = None)
            numPlayoffGames = len(playoffgames)
            teamEloRatings = CalcEloRatings(k, w, games, numGames)
            yearCount += 1
            totYearBrierScore += PlayoffsBrierScore(teamEloRatings, playoffgames, numPlayoffGames, k, w, k1, k2, k3)
            avgYearBrierScore = totYearBrierScore/yearCount
        AddBrierScoreData(avgYearBrierScore, BrierScoreData, k, w, k1, k2, k3)
        if BrierScoreData.shape[0] == 200:
            BrierScoreData = BrierScoreData.sort_values(by='brier')
            BrierScoreData = BrierScoreData.head(20)

In [37]:
BrierScoreData = BrierScoreData.sort_values(by='brier')
BrierScoreData.head(20)
#	0.229336	31.0	-90.0

,brier,K,homeWeight,Round1_K,Round2_K,Round3_K
79,0.230185,65.0,"(79,)",65.0,68.0,65.0
154,0.230185,65.0,"(79,)",66.0,66.0,65.0
54,0.230185,65.0,"(79,)",65.0,67.0,65.0
179,0.230185,65.0,"(79,)",66.0,67.0,65.0
29,0.230185,65.0,"(79,)",65.0,66.0,65.0
104,0.230185,65.0,"(79,)",65.0,69.0,65.0
4,0.230185,65.0,"(79,)",65.0,65.0,65.0
129,0.230185,65.0,"(79,)",66.0,65.0,65.0
24,0.230185,65.0,"(79,)",66.0,68.0,65.0
128,0.230187,65.0,"(78,)",66.0,65.0,65.0


In [32]:
BrierScoreData.to_csv('data/Step3KData', encoding='utf-8')

### Sort and print the ranking of teams

In [42]:
iSort = np.argsort(-eloRatings)

print('\n\n************** ELO Rating Method **************\n')
print('===========================')
print('Rank   Rating      Team   ')
print('===========================')
for i in range(numTeams):
    print(f'{i+1:4d}   {eloRatings[iSort[i]]:.5f}  {teamNames.loc[iSort[i],1]}')

print('')   # extra carriage return



************** ELO Rating Method **************

Rank   Rating      Team   
   1   94.87570   Kansas_City
   2   87.41350   Buffalo
   3   73.45080   Green_Bay
   4   70.96331   New_Orleans
   5   65.75420   Seattle
   6   62.00281   Pittsburgh
   7   53.80376   Tennessee
   8   49.75814   Indianapolis
   9   49.41663   Baltimore
  10   47.17854   Cleveland
  11   44.63020   Tampa_Bay
  12   37.53368   Miami
  13   23.01814   LA_Rams
  14   -6.42118   Arizona
  15   -7.04988   Chicago
  16   -9.89536   LA_Chargers
  17   -10.43522   Las_Vegas
  18   -10.75541   Washington
  19   -13.11933   Minnesota
  20   -13.20343   New_England
  21   -29.24559   San_Francisco
  22   -30.44234   NY_Giants
  23   -31.83675   Dallas
  24   -48.23046   Denver
  25   -49.91004   Carolina
  26   -52.40248   Atlanta
  27   -56.54264   Philadelphia
  28   -59.25077   Cincinnati
  29   -62.76759   Detroit
  30   -68.05702   Houston
  31   -85.86021   NY_Jets
  32   -114.37371   Jacksonville



### Calculate predictability of the method

In [7]:
numberCorrectPredictions = 0
for i in range(numGames):
    team1ID = games.loc[i, 2] - 1 
    team1Score = games.loc[i, 4]
    team2ID = games.loc[i, 5] - 1 
    team2Score = games.loc[i, 7]
    
    if team1Score > team2Score and eloRatings[team1ID] > eloRatings[team2ID]:
        numberCorrectPredictions += 1
    elif team2Score > team1Score and eloRatings[team2ID] > eloRatings[team1ID]:
        numberCorrectPredictions += 1
    elif team1Score == team2Score and eloRatings[team1ID] == eloRatings[team2ID]:
        numberCorrectPredictions += 1

print(f'Predictability: {numberCorrectPredictions/numGames*100:.2f}%') 


Predictability: 75.39%


In [9]:
# #divisions
# NFCdivisions = {'north': ["Chicago", "Detroit","Green_Bay","Minnesota"], 'south': ["Atlanta", "Carolina", "New_Orleans", "Tampa_Bay"], 'east': ["Dallas", "NY_Giants", "Philadelphia", "Washington"], 'west': ["Arizona", "LA_Rams", "San_Francisco", "Seattle"]}
# AFCdivisions = {'north': ["Baltimore", "Cincinnati", "Cleveland", "Pittsburgh"], 'south': ["Houston", "Indianapolis", "Jacksonville", "Tennessee"], 'east': ["Buffalo", "Miami", "New_England", "NY_Jets", ], 'west': ["Denver", "Kansas_City", "Las_Vegas", "LA_Chargers"]}
# nfc = pd.DataFrame(data=NFCdivisions)
# afc = pd.DataFrame(data=AFCdivisions)
# #NFCdivisions + AFCdivisions
# #standings
# wins = np.zeros(numTeams)
# losses = np.zeros(numTeams)
# ties = np.zeros(numTeams)
# winsties = np.zeros(numTeams)

# for i in range(numGames):
#     team1ID = games.loc[i, 2] - 1 # subtracting 1 since python indexes at 0
#     team1Score = games.loc[i, 4]
#     team2ID = games.loc[i, 5] - 1 # subtracting 1 since python indexes at 0
#     team2Score = games.loc[i, 7]

#     if team1Score > team2Score:
#         wins[team1ID] = wins[team1ID]+1
#         winsties[team1ID] = winsties[team1ID]+1
#         losses[team2ID] = losses[team2ID]+1
#     elif team1Score < team2Score:
#         wins[team2ID] = wins[team2ID]+1
#         winsties[team2ID] = winsties[team2ID]+1
#         losses[team1ID] = losses[team1ID]+1
#     else:  
#         winsties[team1ID] = winsties[team1ID]+.5
#         ties[team1ID] = ties[team1ID]+1
#         winsties[team2ID] = winsties[team2ID]+.5
#         ties[team2ID] = ties[team2ID]+1
# #df = pd.DataFrame(wins, columns = ['teamID','teams','wins','losses','ties'])
# winsdf = pd.DataFrame(wins, columns = ['wins'])
# lossesdf = pd.DataFrame(losses, columns = ['losses'])
# tiesdf = pd.DataFrame(ties, columns = ['ties'])
# winstiesdf = pd.DataFrame(winsties, columns = ['winsties'])
# standings = pd.concat([teamNames, winsdf, lossesdf, tiesdf, winstiesdf], axis=1)
# standings.rename(columns={1: "team_name"}, inplace = True)
# standings["conference"] = ""
# namesList = standings["team_name"].to_list()
# newNames = []
# for i in namesList:
#     hi = i.strip()
#     newNames.append(hi)
# #print(newNames)
# standings["team_name"] = newNames
# conferencelist = []
# divisionlist = []
# for i in newNames:
#     for conference, team in NFCdivisions.items():  # for name, age in dictionary.iteritems():  (for Python 2.x)
#         #print(conference)
#         #print(team)
#         if i in team:
#             divisionlist.append("nfc")
#             conferencelist.append(conference)
#     for conference, team in AFCdivisions.items():  # for name, age in dictionary.iteritems():  (for Python 2.x)
#         if i in team:
#             conferencelist.append(conference)
#             divisionlist.append("afc")
# standings["division"] = conferencelist 
# standings["conference"] = divisionlist  
# standings


# #for i in range(32):
#     #name = standings["team_name"].values[i]
#     #if nfc['north'].str.contains(name).any():
#         #print("hi")
#     #for (columnName, columnData) in nfc.iteritems():
#         #for j in range(4):
#             #curTeamName = columnData.values[j]
#             #if standings["team_name"].values[i].str.contains(curTeamName):
#                 #standings.at[i, "confrence"] = "nfc"
#             #else:
#         #standings["confrence"].iloc[i] = "afc"
#                 #standings.at[i, "confrence"] = "afc"
# #standings
# #test = standings[~standings["team_name"].isin(['Houston'])]
# #test = standings[~standings["team_name"].str.contains("Houston")]
# #test = standings[(standings["team_name"].__eq__('Houston'))]
# #print(test)

,0,team_name,wins,losses,ties,winsties,conference,division
0,1,Arizona,8.0,8.0,0.0,8.0,nfc,west
1,2,Atlanta,4.0,12.0,0.0,4.0,nfc,south
2,3,Baltimore,11.0,5.0,0.0,11.0,afc,north
3,4,Buffalo,13.0,3.0,0.0,13.0,afc,east
4,5,Carolina,5.0,11.0,0.0,5.0,nfc,south
5,6,Chicago,8.0,8.0,0.0,8.0,nfc,north
6,7,Cincinnati,4.0,11.0,1.0,4.5,afc,north
7,8,Cleveland,11.0,5.0,0.0,11.0,afc,north
8,9,Dallas,6.0,10.0,0.0,6.0,nfc,east
9,10,Denver,5.0,11.0,0.0,5.0,afc,west


In [13]:
# #playoff bracket
# nfcplayoffs = pd.DataFrame(data = {'team_name': []})
# afcplayoffs = pd.DataFrame(data = {'team_name': []})
# nfcTempStandings = standings[(standings["conference"].str.contains("nfc"))]
# afcTempStandings = standings[(standings["conference"].str.contains("afc"))]
# #print(tempStandings)

# for (columnName, columnData) in nfc.iteritems():
#     playoffTeam = ""
#     playoffTeamWinTies = 0
#     for i in range(4):
#         curTeamName = columnData.values[i]
#         teamInfo = standings[(standings["team_name"].str.contains(curTeamName))]
#         if teamInfo['winsties'].iloc[0] > playoffTeamWinTies:
#             playoffTeamWinTies = teamInfo['winsties'].iloc[0]
#             playoffTeam = teamInfo["team_name"].iloc[0]
#     nfcTempStandings = nfcTempStandings[~nfcTempStandings["team_name"].str.contains(playoffTeam)]
#     tempdf = pd.DataFrame(data = {'team_name': [playoffTeam]})
#     nfcplayoffs = pd.concat([nfcplayoffs, tempdf])
    
# nfcTempStandings = nfcTempStandings.sort_values(by='winsties', ascending=False) 
# nfcwildcard = pd.DataFrame(data = {'team_name': [nfcTempStandings["team_name"].values[0], nfcTempStandings["team_name"].values[1]]})
# nfcplayoffs = pd.concat([nfcplayoffs, nfcwildcard])
# nfcplayoffs= nfcplayoffs.merge(standings, on='team_name', how='left')
# print(nfcplayoffs)

# for (columnName, columnData) in afc.iteritems():
#     playoffTeam = ""
#     playoffTeamWinTies = 0
#     for i in range(4):
#         curTeamName = columnData.values[i]
#         teamInfo = standings[(standings["team_name"].str.contains(curTeamName))]
#         if teamInfo['winsties'].iloc[0] > playoffTeamWinTies:
#             playoffTeamWinTies = teamInfo['winsties'].iloc[0]
#             playoffTeam = teamInfo["team_name"].iloc[0]
#     afcTempStandings = afcTempStandings[~afcTempStandings["team_name"].str.contains(playoffTeam)]
#     tempdf = pd.DataFrame(data = {'team_name': [playoffTeam]})
#     afcplayoffs = pd.concat([afcplayoffs, tempdf])
    
# afcTempStandings = afcTempStandings.sort_values(by='winsties', ascending=False) 
# afcwildcard = pd.DataFrame(data = {'team_name': [afcTempStandings["team_name"].values[0], afcTempStandings["team_name"].values[1]]})
# afcplayoffs = pd.concat([afcplayoffs, afcwildcard])
# afcplayoffs= afcplayoffs.merge(standings, on='team_name', how='left')
# print(afcplayoffs)

     team_name   0  wins  losses  ties  winsties conference division
0    Green_Bay  12  13.0     3.0   0.0      13.0        nfc    north
1  New_Orleans  23  12.0     4.0   0.0      12.0        nfc    south
2   Washington  32   7.0     9.0   0.0       7.0        nfc     east
3      Seattle  29  12.0     4.0   0.0      12.0        nfc     west
4    Tampa_Bay  30  11.0     5.0   0.0      11.0        nfc    south
5      LA_Rams  18  10.0     6.0   0.0      10.0        nfc     west
      team_name   0  wins  losses  ties  winsties conference division
0    Pittsburgh  27  12.0     4.0   0.0      12.0        afc    north
1  Indianapolis  14  11.0     5.0   0.0      11.0        afc    south
2       Buffalo   4  13.0     3.0   0.0      13.0        afc     east
3   Kansas_City  16  14.0     2.0   0.0      14.0        afc     west
4     Baltimore   3  11.0     5.0   0.0      11.0        afc    north
5     Cleveland   8  11.0     5.0   0.0      11.0        afc    north
